# The Outer Loop

## Read Links and Store in Dictionary

In [1]:
#!pip install xlrd
#!pip install selenium

In [226]:
test = True
test_paragraph = "XaXeixoxu XaXxx"

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from constants import COUNTERS

In [3]:
from bs4 import BeautifulSoup

from selenium import webdriver

In [4]:
import pandas as pd
df_work = pd.read_excel('Input file.xlsx')
df_work.head()

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


## Extracting header tag using Selenium

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

In [6]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from lxml import etree
import requests

options = Options()
options.add_argument('--headless')
options.add_argument("--windows-size=1920,1200")

driver = webdriver.Chrome()    


for i in range(len(df_work)):
#for i in range(5):
    link = df_work.loc[i,"URL"]
    try:
        driver.get(link)
            #df_work["title"] = driver.title
        df_work.loc[i, "title"] = ""
        df_work.loc[i, "title"] = driver.title

        df_work.loc[i, "article"] = ""

        article = driver.find_elements(By.XPATH, '//p[not(@*)]')  # All <p> tags with no attributes.
        for paragraph in article:            # Looking to concatenate all strings in array

            df_work.loc[i, "article"] = " ".join([df_work.loc[i, "article"], paragraph.text])   #df_work.loc[i, "article"] + paragraph.text  
    except: 
         continue
driver.quit()
     

In [7]:
df_work.head()

,URL_ID,URL,title,article
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes -...,Introduction “If anything kills over 10 milli...
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creato...,"Human minds, a fascination in itself carrying..."
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction AI is rapidly evolving in the em...
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-tha...
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us? - Blackcof...,“Machine intelligence is the last invention t...


### Now the next couple of steps are stemming, stopwords, Positive/Negative keywords and statistics

#### Write to File

In [8]:
for idx in df_work.index:
    text_file = open("./Files/"+str(df_work["URL_ID"][idx]), "w")
    text_file.write(str(df_work["article"][idx]))
    text_file.close()

For each article
1. Loop through all files and remove stop words

## Files for Stopwords

In [9]:
#to populate 'stopwords' variable
#STOPWORDS_FILES = ["StopWords_DatesandNumbers.txt",]
STOPWORDS_FILES=["StopWords_Auditor.txt", "StopWords_Currencies.txt", "StopWords_DatesandNumbers.txt",
                 "StopWords_Generic.txt", "StopWords_GenericLong.txt", "StopWords_Geographic.txt",
                 "StopWords_Names.txt"]
stopwords = [] #"million"
#Check next line
stopwords_array = []
for file in STOPWORDS_FILES:
    temp_file = open('StopWords/' + file, "r")
    stopwords_array.append(temp_file.readlines())
    temp_file.close()

for stopwords_list in stopwords_array:
    for stopword in stopwords_list:
        stopwords.append(stopword.strip().lower())

In [156]:
#The latest and greatest
processed_articles = []
for idx in df_work.index:
    article = df_work.loc[idx,"article"]

    if(test):
        article = test_paragraph

    if(type(article) is str):
        article_words = [word.lower() for word in article.split()]
        for stopword in stopwords:       
            for word in article_words:
                if(stopword == word):
                    article_words.remove(stopword)
        article = " ".join(article_words)
        processed_articles.append(article)

['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china', 'on', 'december']
['in', 'china'

In [157]:
processed_articles[113]

''

## Positive/Negative Word Dictionaries

In [163]:
#Read all positive words and output them one by one
positive_score = [0]*113
file = open('positive-words.txt','r')
positive_words = file.readlines()
file.close()

#print(positive_words)
positive_words = [s.strip() for s in positive_words]
    
for idx in range(113): 
    words = processed_articles[idx].split()
    
    if(test):
        words = test_paragraph.split()

#    print(words)
    for word in words:
        if (word in positive_words):
            positive_score[idx] += 1 
            
print(positive_score)


[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


In [166]:
#Read all negative words and output them one by one
negative_score = [0]*113
file = open('negative-words.txt', 'r')
negative_words = file.readlines()
file.close()

negative_words = [s.strip() for s in negative_words]

for idx in range(113):
    words = processed_articles[idx].split()
    
    if(test):
        words = test_paragraph.split()
    
    for word in words:
        if (word in negative_words):
            negative_score[idx] += 1 
            
print(negative_score)

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [167]:
#Polarity score
polarity_score = [0]*113
for idx in range(113):
    polarity_score[idx] = (positive_score[idx] - negative_score[idx])/(positive_score[idx] + negative_score[idx] + 0.000001)
print(polarity_score)

[0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825, 0.1428571224489825,

In [168]:
#Subjectivity score
subjectivity_score = [0]*113
for idx in range(113):
    subjectivity_score[idx] = (positive_score[idx] + negative_score[idx])/(len(processed_articles[idx]) + 0.000001)
print(subjectivity_score)

[7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0, 7000000.0

In [203]:
#average sentence length

average_sentence_length = [0]*113
word_count = [0]*113

for idx in range(113):
    sentences = processed_articles[idx][0:-1].split('.')
    
    if(test):
        sentences = test_paragraph[0:-1].split('.')
        
    
#    print(sentences) 
    
    
    for sentence in sentences:
        if(sentence == ''):
            break
        words = sentence.split()
        word_count[idx] += len(words)

    average_sentence_length[idx] = (word_count[idx])/len(sentences)
print(average_sentence_length)

['SENTENCE OF LENGHT 4', ' SENTENCE OF LENGTH ONLY 5', ' SENTENCE OF LENGHT ONLY ONLY 6']
['SENTENCE OF LENGHT 4', ' SENTENCE OF LENGTH ONLY 5', ' SENTENCE OF LENGHT ONLY ONLY 6']
['SENTENCE OF LENGHT 4', ' SENTENCE OF LENGTH ONLY 5', ' SENTENCE OF LENGHT ONLY ONLY 6']
['SENTENCE OF LENGHT 4', ' SENTENCE OF LENGTH ONLY 5', ' SENTENCE OF LENGHT ONLY ONLY 6']
['SENTENCE OF LENGHT 4', ' SENTENCE OF LENGTH ONLY 5', ' SENTENCE OF LENGHT ONLY ONLY 6']
['SENTENCE OF LENGHT 4', ' SENTENCE OF LENGTH ONLY 5', ' SENTENCE OF LENGHT ONLY ONLY 6']
['SENTENCE OF LENGHT 4', ' SENTENCE OF LENGTH ONLY 5', ' SENTENCE OF LENGHT ONLY ONLY 6']
['SENTENCE OF LENGHT 4', ' SENTENCE OF LENGTH ONLY 5', ' SENTENCE OF LENGHT ONLY ONLY 6']
['SENTENCE OF LENGHT 4', ' SENTENCE OF LENGTH ONLY 5', ' SENTENCE OF LENGHT ONLY ONLY 6']
['SENTENCE OF LENGHT 4', ' SENTENCE OF LENGTH ONLY 5', ' SENTENCE OF LENGHT ONLY ONLY 6']
['SENTENCE OF LENGHT 4', ' SENTENCE OF LENGTH ONLY 5', ' SENTENCE OF LENGHT ONLY ONLY 6']
['SENTENCE

In [206]:
### Complex Word Count

complex_word_count = [0]*113
for idx in range(113):
    sentences = processed_articles[idx].split('.')
    
    if(test):
        sentences = test_paragraph.split('.')
    
    for sentence in sentences:
        words = sentence.split()
        for word in words:
            char_count = 0
            for char in word:
                if(char == 'a' or char == 'e' or char == 'i' or char == 'o' or char =='u'):
                    char_count+=1
            if(char_count > 2):
                complex_word_count[idx] += 1
print(complex_word_count)

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [76]:
## Already got word count from 'averate sentence length' cell

In [228]:
## Syllable count per word
syllable_count = [0]*113
word_count_articles = [0]*113
 
for idx in range(113):
    words = processed_articles[idx].split()
    
    if(test):
        words = test_paragraph.split()
    
    print(words)
    
    for word in words:
        syllable_count_word = 0
        for char in word:
                if(char == 'a' or char == 'e' or char == 'i' or char == 'o' or char =='u'):
                    syllable_count_word +=1
        word_count_articles[idx] +=1 
        syllable_count[idx] = syllable_count[idx] + syllable_count_word
    
                    
    if(word_count_articles[idx]!= 0):
        print(syllable_count[idx]/word_count_articles[idx])


['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
['XaXeixoxu', 'XaXxx']
3.0
[

In [210]:
#Personal pronouns count “I,” “we,” “my,” “ours,” and “us”.

pronoun_count = [0]*113
for idx in range(113):
    words = processed_articles[idx].split()
    
    if(test):
        words = test_paragraph.split()
    
    
    for word in words:
        #print(word)
        if (word == 'I' or word == 'we' or word == 'my' or word == 'ours' or word == 'us'):
            pronoun_count[idx] += 1

print(pronoun_count) #0 as stopwords had I, we etc

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


In [212]:
#Average word length
#Sum of the total number of characters in each word/Total number of words

word_count = [0]*113
char_count = [0]*113
avg_word_len = [0]*113
for idx in range(113):
    words = processed_articles[idx].split()
    
    if(test):
        words = test_paragraph.split()
    
    for word in words:
        char_count[idx] += len(word)
        word_count[idx] += 1
    if(word_count[idx]!=0):
        avg_word_len[idx]+=char_count[idx]/word_count[idx]


print(avg_word_len)

[2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75, 2.75]


### Creating Output File Structure

In [ ]:
#!pip install xlwt

In [114]:
df_work.loc[1, "URL_ID"]

38

In [120]:
import xlwt
from xlwt import Workbook


df_url_id = df_work["URL_ID"]
df_url = df_work["URL"]


wb = Workbook()

output_sheet = wb.add_sheet('Output')

output_sheet.write(0, 1, 'URL_ID')
output_sheet.write(0, 2, 'URL')
output_sheet.write(0, 3, 'POSITIVE SCORE')
output_sheet.write(0, 4, 'NEGATIVE SCORE')
output_sheet.write(0, 5, 'POLARITY SCORE')
output_sheet.write(0, 6, 'SUBJECTIVITY SCORE')
output_sheet.write(0, 7, 'AVG SENTENCE LENGTH')
output_sheet.write(0, 8, 'PERCENTAGE OF COMPLEX WORDS')
output_sheet.write(0, 9, 'FOG INDEX')
output_sheet.write(0, 10, 'AVG NUMBER OF WORDS PER SENTENCE')
output_sheet.write(0, 11, 'COMPLEX WORD COUNT')
output_sheet.write(0, 12, 'WORD COUNT')
output_sheet.write(0, 13, 'SYLLABLE PER WORD')
output_sheet.write(0, 14, 'PERSONAL PRONOUNS')
output_sheet.write(0, 15, 'AVG WORD LENGTH')

for idx in range(1, 113):
    output_sheet.write(idx, 1, str(df_work.loc[idx, 'URL_ID']))
    output_sheet.write(idx, 2, df_work.loc[idx, 'URL'])
    output_sheet.write(idx, 3, str(positive_score[idx]))
    output_sheet.write(idx, 4, str(negative_score[idx]))
    output_sheet.write(idx, 5, str(polarity_score[idx]))
    output_sheet.write(idx, 6, str(subjectivity_score[idx]))
    output_sheet.write(idx, 7, str(average_sentence_length[idx]))
    output_sheet.write(idx, 8, str(complex_word_count[idx]/word_count))
    output_sheet.write(idx, 9, str(0.4()))
    output_sheet.write(idx, 10, 'AVG NUMBER OF WORDS PER SENTENCE')
    output_sheet.write(idx, 11, str(complex_word_count[idx]))
    output_sheet.write(idx, 12, str(word_count))
    output_sheet.write(idx, 13, 'SYLLABLE PER WORD')
    output_sheet.write(idx, 14, str(pronoun_count[idx]))
    output_sheet.write(idx, 15, str(avg_word_len[idx]))

#for i in range(len(df_work)):
# for i in range(10):
#     output_sheet.write(i+1, 1, str(df_url_id[i]))
#     output_sheet.write(i+1, 2, df_url[i]) #i+1 because at i=0, the heading labels are defined

wb.save('output.xls')